Ленивая функция, возвращает результат по запросу:

In [2]:
map(ord, "The quick brown fox")

In [7]:
print(list(map(ord, "The quick brown fox")))

[84, 104, 101, 32, 113, 117, 105, 99, 107, 32, 98, 114, 111, 119, 110, 32, 102, 111, 120]


![map](https://raw.githubusercontent.com/dm-fedorov/advanced-python/master/pic/pic_01.jpg)

In [10]:
for o in map(ord, "The quick brown fox"):
    print(o)

84
104
101
32
113
117
105
99
107
32
98
114
111
119
110
32
102
111
120


Создадим ручной трассировщик:

In [3]:
class Trace:
    def __init__(self):
        self.enabled = True

    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self.enabled:
                print('Calling {}'.format(f))
            return f(*args, **kwargs)
        return wrap

In [4]:
result = map(Trace()(ord), "The quick brown fox")

In [5]:
result

In [8]:
next(result)

Calling <built-in function ord>


84

In [9]:
next(result)

Calling <built-in function ord>


104

Списки в качестве аргументов:

![map2](https://raw.githubusercontent.com/dm-fedorov/advanced-python/master/pic/pic_02.jpg)

In [11]:
m1 = ['1', '2', '3']
m2 = ['4', '5', '6']
m3 = ['7', '8', '9']

In [12]:
def combine(m1, m2, m3):
    return f"{m1} {m2} {m3}"

In [13]:
list(map(combine, m1, m2, m3))

['1 4 7', '2 5 8', '3 6 9']

In [16]:
import itertools
def combine(m0, m1, m2, m3):
    return f"{m0} -> {m1} {m2} {m3}"

In [17]:
list(map(combine, itertools.count(), m1, m2, m3))

['0 -> 1 4 7', '1 -> 2 5 8', '2 -> 3 6 9']

![filter](https://raw.githubusercontent.com/dm-fedorov/advanced-python/master/pic/pic_03.jpg)

In [18]:
positive = filter(lambda x: x > 0, [1, -5, 0, 6, -2, 8])
positive

In [19]:
list(positive)

[1, 6, 8]

Если передать None в качестве первого аргумента в [filter()](https://docs.python.org/3/library/functions.html#filter), то будут удалены ложные (False) элементы:

In [20]:
true = filter(None, [0, 1, False, True, [], [1, 2, 3], '', 'hello'])
list(true)

[1, True, [1, 2, 3], 'hello']

Функциональные возможности собраны в модуле [functools](https://docs.python.org/3/library/functools.html#functools.reduce):

In [21]:
from functools import reduce
import operator 
reduce(operator.add, [1, 2, 3, 4, 5])

15

In [22]:
numbers = [1, 2, 3, 4, 5]
accumulator = operator.add(numbers[0], numbers[1])
for item in numbers[2:]:
    accumulator = operator.add(accumulator, item)

In [23]:
accumulator

15

[Эквивалентно](https://github.com/python/cpython/blob/3.8/Lib/functools.py):

In [ ]:
def reduce(function, iterable, initializer=None):
    it = iter(iterable)
    if initializer is None:
        value = next(it)
    else:
        value = initializer
    for element in it:
        value = function(value, element)
    return value

In [24]:
def mul(x, y):
    print(f"mul {x} {y}")
    return x * y

In [25]:
reduce(mul, range(1, 10))

mul 1 2
mul 2 3
mul 6 4
mul 24 5
mul 120 6
mul 720 7
mul 5040 8
mul 40320 9


362880

In [26]:
reduce(mul, [])

TypeError: reduce() of empty sequence with no initial value

In [27]:
reduce(mul, [1])

1

### Разбираемся с map reduce

In [28]:
documents = [
    'It was the best of times, it was the worst of times.',
    'I went to the woods because I wished to live deliberately, to front only the essential facts of life...',
    'Friends, Romans, countrymen, lend me your ears; I come to bury Caesar, not to praise him.',
    'I do not like green eggs and ham. I do not like them, Sam-I-Am.'
]

In [29]:
def count_words(doc):
    normalised_doc = ''.join(c.lower() if c.isalpha() else ' ' for c in doc)
    frequencies = {}
    for word in normalised_doc.split():
        frequencies[word] = frequencies.get(word, 0) + 1
    return frequencies

In [30]:
counts = map(count_words, documents)

In [31]:
def combine_counts(d1, d2):
    d = d1.copy()
    for word, count in d2.items():
        d[word] = d.get(word, 0) + count
    return d

In [32]:
from functools import reduce

total_counts = reduce(combine_counts, counts)

In [35]:
print(total_counts)

{'it': 2, 'was': 2, 'the': 4, 'best': 1, 'of': 3, 'times': 2, 'worst': 1, 'i': 6, 'went': 1, 'to': 5, 'woods': 1, 'because': 1, 'wished': 1, 'live': 1, 'deliberately': 1, 'front': 1, 'only': 1, 'essential': 1, 'facts': 1, 'life': 1, 'friends': 1, 'romans': 1, 'countrymen': 1, 'lend': 1, 'me': 1, 'your': 1, 'ears': 1, 'come': 1, 'bury': 1, 'caesar': 1, 'not': 3, 'praise': 1, 'him': 1, 'do': 2, 'like': 2, 'green': 1, 'eggs': 1, 'and': 1, 'ham': 1, 'them': 1, 'sam': 1, 'am': 1}


In [36]:
class ExampleIterator:
    def __init__(self, data):
        self.index = 0
        self.data = data

    def __iter__(self):
        return self

    def __next__(self):
        if self.index >= len(self.data):
            raise StopIteration()

        rslt = self.data[self.index]
        self.index += 1
        return rslt

class ExampleIterable:
    def __init__(self):
        self.data = [1, 2, 3]

    def __iter__(self):
        return ExampleIterator(self.data)


In [38]:
for i in ExampleIterable():
    print(i)

1
2
3


In [39]:
[i * 3 for i in ExampleIterable()]

[3, 6, 9]

In [40]:
class AlternateIterable:
    def __init__(self):
        self.data = [1, 2, 3]

    def __getitem__(self, idx):
        return self.data[idx]

In [41]:
[i for i in AlternateIterable()]

[1, 2, 3]

Расширенный iter() часто используется для создания бесконечных последовательностей из существующих функций.

In [42]:
import datetime 
i = iter(datetime.datetime.now, None)

In [43]:
next(i)

datetime.datetime(2020, 9, 8, 14, 18, 31, 131628)

In [44]:
next(i)

datetime.datetime(2020, 9, 8, 14, 18, 36, 453446)

In [45]:
next(i)

datetime.datetime(2020, 9, 8, 14, 18, 44, 124234)

In [46]:
!cat data/ending_file.txt

You should
see this
text.
END
But not
this text.


In [47]:
with open("data/ending_file.txt", "rt") as f:
    for line in iter(lambda: f.readline().strip(), "END"):
        print(line)

You should
see this
text.


Логирование показаний какого-нибудь сенсора (температура, давление, радиация и пр.):

In [48]:
import datetime
import itertools
import random
import time

class Sensor:
    def __iter__(self):
        return self

    def __next__(self):
        return random.random()

sensor = Sensor()
timestamps = iter(datetime.datetime.now, None)

for stamp, value in itertools.islice(zip(timestamps, sensor), 10):
    print(stamp, value)
    time.sleep(1)

2020-09-08 14:22:45.229723 0.586131718340358
2020-09-08 14:22:46.231082 0.5706658688584275
2020-09-08 14:22:47.232290 0.04151088864770747
2020-09-08 14:22:48.233407 0.4662469337489459


KeyboardInterrupt: 

In [ ]:
class CompressedList(list):
    def __iter__(self):
        return ComressedListIterator(self)

class ComressedListIterator:
    def __init__(self, arr):
        self.arr = arr
        self.counter = 0
        self.index = 0
  
    def __next__(self):
        # если все пары перебрали, raise StopIteration()
        
        # если не все повторяющиеся элементы 
        # текущей пары self.index выдали - увеличиваем counter
        # выдаем еще один элемент
    
        # если все элементы текущей пары выдали, переходим к 
        # следующей паре

original = [1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 3, 3, 3]
compressed = CompressedList([(1, 4), (2, 2), (1, 3), (3, 4)])

decompressed = [x for x in compressed]

print(original)
print(decompressed)
print(original == decompressed)

In [ ]:
class CompressedList(list):
    def __iter__(self):
        return ComressedListIterator(self)

class ComressedListIterator:
    def __init__(self, arr):
        self.arr = arr
        self.counter = 0
        self.index = 0
  
    def __next__(self):
        # если все пары перебрали, raise StopIteration()
        
        # если не все повторяющиеся элементы 
        # текущей пары self.index выдали - увеличиваем counter
        # выдаем еще один элемент
    
        # если все элементы текущей пары выдали, переходим к 
        # следующей паре

original = [1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 3, 3, 3]
compressed = CompressedList([(1, 4), (2, 2), (1, 3), (3, 4)])

decompressed = [x for x in compressed]

print(original)
print(decompressed)
print(original == decompressed)